<a href="https://colab.research.google.com/github/aramasethu/CS_573_project/blob/main/Merge_DM_Aishwarya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting google drive

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Mobility data

In [ ]:
mobility = pd.read_csv (r'/content/drive/MyDrive/CS_573_Group_project/Data/Global_Mobility_Report.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (3,4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
mobility_country1 = mobility[mobility['sub_region_1'].isna()]
mobility_country = mobility_country1[mobility_country1['metro_area'].isna()]

In [ ]:
mobility_country = mobility_country.drop(columns=['country_region_code','sub_region_1', 'sub_region_2','metro_area', 'iso_3166_2_code','census_fips_code','place_id'])

In [ ]:
mobility_country['date'] = mobility_country['date'].astype('datetime64[ns]')

In [ ]:
mobility_country = mobility_country.rename(columns={ 'country_region': 'country'})

## Vaccination data loading

In [ ]:
vec_cases = pd.read_csv(r'/content/drive/MyDrive/CS_573_Group_project/Data/owid-covid-data.csv')

In [ ]:
vec_cases_scraped = vec_cases[['location', 'date', 'total_cases', 'total_deaths', 'population',
                              'new_vaccinations_smoothed', 'new_people_vaccinated_smoothed']]

In [ ]:
vec_cases_new = vec_cases_scraped.copy()

In [ ]:
vec_cases_new['total_cases'] = vec_cases_new['total_cases'].replace(np.nan, 0)
vec_cases_new['total_deaths'] = vec_cases_new['total_deaths'].replace(np.nan, 0)
vec_cases_new['new_vaccinations_smoothed'] = vec_cases_new['new_vaccinations_smoothed'].replace(np.nan, 0)
vec_cases_new['new_people_vaccinated_smoothed'] = vec_cases_new['new_people_vaccinated_smoothed'].replace(np.nan, 0)

In [ ]:
vec_cases_new['date'] = pd.to_datetime(vec_cases_new['date']).dt.strftime('%Y/%m/%d')

In [ ]:
vec_cases_new = vec_cases_new.rename(columns={'location': 'country'})

In [ ]:
vec_cases_new['date'] = vec_cases_new['date'].astype('datetime64[ns]')

## Lockdown data

In [ ]:
lockdown = pd.read_excel (r'/content/drive/MyDrive/CS_573_Group_project/Data/OxCGRT_timeseries_all.xlsx')

In [ ]:
lockdown_new = lockdown.drop(columns={"country_name","country_code"})
lockdown_new.columns = pd.to_datetime(lockdown_new.columns)

In [50]:
lockdown_input = pd.DataFrame(columns=["country_name","lockdown_index","date"])
for i in range(lockdown_new.shape[1]):
  country = lockdown[{"country_name"}]
  index = lockdown_new.iloc[:,i]
  index_con = pd.concat([country,index], axis=1, ignore_index=True)
  index_con["date"] = index.name
  index_con = index_con.rename(columns={0: 'country_name',  1: 'lockdown_index'})
  lockdown_input = pd.concat([lockdown_input,index_con],ignore_index=True)

In [51]:
lockdown_input = lockdown_input.sort_values('country_name',ascending=True)

In [52]:
lockdown_input = lockdown_input.rename(columns={"country_name":"country"})

## Merge all datasets

In [53]:
df_merged1 = vec_cases_new.merge(mobility_country, on=['country', 'date'])

In [54]:
df_merged1.shape

(97134, 13)

In [55]:
df_merged = df_merged1.merge(lockdown_input, on=['country', 'date'])

In [56]:
df_merged.shape

(92719, 14)

In [57]:
df_merged.head()

,country,date,total_cases,total_deaths,population,new_vaccinations_smoothed,new_people_vaccinated_smoothed,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,lockdown_index
0,Afghanistan,2020-02-24,5.0,0.0,39835428.0,0.0,0.0,3.0,13.0,4.0,9.0,7.0,0.0,8.33
1,Afghanistan,2020-02-25,5.0,0.0,39835428.0,0.0,0.0,2.0,5.0,1.0,9.0,7.0,0.0,8.33
2,Afghanistan,2020-02-26,5.0,0.0,39835428.0,0.0,0.0,-1.0,5.0,2.0,3.0,5.0,1.0,8.33
3,Afghanistan,2020-02-27,5.0,0.0,39835428.0,0.0,0.0,-1.0,0.0,1.0,3.0,7.0,2.0,8.33
4,Afghanistan,2020-02-28,5.0,0.0,39835428.0,0.0,0.0,-2.0,5.0,1.0,4.0,5.0,2.0,8.33
